In [25]:
!git clone https://github.com/dhyougit/keras-wtte-rnn.git

Cloning into 'keras-wtte-rnn'...
remote: Enumerating objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (32/32), done.


In [26]:
!ls

data_readme.txt  license.txt  test_x.csv  train.csv
keras-wtte-rnn	 readme.md    test_y.csv  wtte-rnn.py


In [27]:
import os
os.listdir()
x = os.listdir("./keras-wtte-rnn")
x

['train.csv',
 'data_readme.txt',
 'test_y.csv',
 'readme.md',
 'license.txt',
 '.git',
 'test_x.csv',
 '.gitignore',
 'wtte-rnn.py']

In [28]:
os.chdir("./keras-wtte-rnn")

In [29]:
os.listdir()

['train.csv',
 'data_readme.txt',
 'test_y.csv',
 'readme.md',
 'license.txt',
 '.git',
 'test_x.csv',
 '.gitignore',
 'wtte-rnn.py']

In [30]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Masking
from keras.optimizers import RMSprop
from keras import backend as k
from sklearn.preprocessing import normalize


In [31]:
"""
    Discrete log-likelihood for Weibull hazard function on censored survival data
    y_true is a (samples, 2) tensor containing time-to-event (y), and an event indicator (u)
    ab_pred is a (samples, 2) tensor containing predicted Weibull alpha (a) and beta (b) parameters
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)

    단절된 생존데이터에 대한 weibull위험 함수의 이산적 로그우도비.
    y_true는 time-to-event값인 텐서 와 이벤트 지표이고 
    ab_pred는 weibull분포의 알파값과 베타값을 예측한 텐서이다. 
    자세한 수학적 기반은 위의 깃에서 확인. 
"""
def weibull_loglik_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    hazard0 = k.pow((y_ + 1e-35) / a_, b_)
    hazard1 = k.pow((y_ + 1) / a_, b_)

    return -1 * k.mean(u_ * k.log(k.exp(hazard1 - hazard0) - 1.0) - hazard1)

In [32]:
""" 연속인경우 아래함수로.... 
    Not used for this model, but included in case somebody needs it
    For math, see https://ragulpr.github.io/assets/draft_master_thesis_martinsson_egil_wtte_rnn_2016.pdf (Page 35)
"""
def weibull_loglik_continuous(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    ya = (y_ + 1e-35) / a_
    return -1 * k.mean(u_ * (k.log(b_) + b_ * k.log(ya)) - k.pow(ya, b_))

In [33]:
"""
   이부분이 묘미. 케라스를 사용해서 아웃풋의 종류를 레이어 별로 다르게 설정해 주는것.. 
    Custom Keras activation function, outputs alpha neuron using exponentiation and beta using softplus
"""
def activate(ab):
    a = k.exp(ab[:, 0])
    b = k.softplus(ab[:, 1])

    a = k.reshape(a, (k.shape(a)[0], 1))
    b = k.reshape(b, (k.shape(b)[0], 1))

    return k.concatenate((a, b), axis=1)

In [34]:
"""
데이터 전처리 과정. 
x끼리 합쳐서 정규화 한다음 다시 나누기.
y에는 100일의 관찰 기관을 채우지 못한 데이터에 한해 0을 채움. 엄밀히 말하면 이부분 때문에  bias가 생길수는 있음.  
    Load and parse engine data files into:
       - an (engine/day, observed history, sensor readings) x tensor, where observed history is 100 days, zero-padded
         for days that don't have a full 100 days of observed history (e.g., first observed day for an engine)
       - an (engine/day, 2) tensor containing time-to-event and 1 (since all engines failed)
    There are probably MUCH better ways of doing this, but I don't use Numpy that much, and the data parsing isn't the
    point of this demo anyway.
"""
def load_file(name):
    with open(name, 'r') as file:
        return np.loadtxt(file, delimiter=',')

np.set_printoptions(suppress=True, threshold=10000)

train = load_file('train.csv')
test_x = load_file('test_x.csv')
test_y = load_file('test_y.csv')

# Combine the X values to normalize them, then split them back out
all_x = np.concatenate((train[:, 2:26], test_x[:, 2:26]))
all_x = normalize(all_x, axis=0)

train[:, 2:26] = all_x[0:train.shape[0], :]
test_x[:, 2:26] = all_x[train.shape[0]:, :]

# Make engine numbers and days zero-indexed, for everybody's sanity
train[:, 0:2] -= 1
test_x[:, 0:2] -= 1

# Configurable observation look-back period for each engine/day
max_time = 100

def build_data(engine, time, x, max_time, is_test):
    # y[0] will be days remaining, y[1] will be event indicator, always 1 for this data
    out_y = np.empty((0, 2), dtype=np.float32)

    # A full history of sensor readings to date for each x
    out_x = np.empty((0, max_time, 24), dtype=np.float32)

    for i in range(100):
        print("Engine: " + str(i))
        # When did the engine fail? (Last day + 1 for train data, irrelevant for test.)
        max_engine_time = int(np.max(time[engine == i])) + 1

        if is_test:
            start = max_engine_time - 1
        else:
            start = 0

        this_x = np.empty((0, max_time, 24), dtype=np.float32)

        for j in range(start, max_engine_time):
            engine_x = x[engine == i]

            out_y = np.append(out_y, np.array((max_engine_time - j, 1), ndmin=2), axis=0)

            xtemp = np.zeros((1, max_time, 24))
            xtemp[:, max_time-min(j, 99)-1:max_time, :] = engine_x[max(0, j-max_time+1):j+1, :]
            this_x = np.concatenate((this_x, xtemp))

        out_x = np.concatenate((out_x, this_x))

    return out_x, out_y

train_x, train_y = build_data(train[:, 0], train[:, 1], train[:, 2:26], max_time, False)
test_x = build_data(test_x[:, 0], test_x[:, 1], test_x[:, 2:26], max_time, True)[0]

train_u = np.zeros((100, 1), dtype=np.float32)
train_u += 1
test_y = np.append(np.reshape(test_y, (100, 1)), train_u, axis=1)

Engine: 0
Engine: 1
Engine: 2
Engine: 3
Engine: 4
Engine: 5
Engine: 6
Engine: 7
Engine: 8
Engine: 9
Engine: 10
Engine: 11
Engine: 12
Engine: 13
Engine: 14
Engine: 15
Engine: 16
Engine: 17
Engine: 18
Engine: 19
Engine: 20
Engine: 21
Engine: 22
Engine: 23
Engine: 24
Engine: 25
Engine: 26
Engine: 27
Engine: 28
Engine: 29
Engine: 30
Engine: 31
Engine: 32
Engine: 33
Engine: 34
Engine: 35
Engine: 36
Engine: 37
Engine: 38
Engine: 39
Engine: 40
Engine: 41
Engine: 42
Engine: 43
Engine: 44
Engine: 45
Engine: 46
Engine: 47
Engine: 48
Engine: 49
Engine: 50
Engine: 51
Engine: 52
Engine: 53
Engine: 54
Engine: 55
Engine: 56
Engine: 57
Engine: 58
Engine: 59
Engine: 60
Engine: 61
Engine: 62
Engine: 63
Engine: 64
Engine: 65
Engine: 66
Engine: 67
Engine: 68
Engine: 69
Engine: 70
Engine: 71
Engine: 72
Engine: 73
Engine: 74
Engine: 75
Engine: 76
Engine: 77
Engine: 78
Engine: 79
Engine: 80
Engine: 81
Engine: 82
Engine: 83
Engine: 84
Engine: 85
Engine: 86
Engine: 87
Engine: 88
Engine: 89
Engine: 90
Engine: 9

In [38]:
"""
모델 피팅. 
1) 모델 시퀀스 틀 만들기 
2) 룩백할 과거 데이터에 0을 채워 주기 (?) 
3) 시간의 순서에 따른 요소들을 신경망에 엮어주기 위해 LSTM을 설정해주기 
4) weibull분포를 위해서 알파와 베타값을 설정해 주기위해 아웃풋에 뉴런 2개를 설정해 주기 
5) 위에서 만든 활성함수를 불러준다 
6) 위에서 만든 웨이블분포를 사용한 생존분석의 이산 로그우도비를 손실함수로 설정해 주기 
7) Fit 
    Here's the rest of the meat of the demo... actually fitting and training the model.
    We'll also make some test predictions so we can evaluate model performance.
"""
model = Sequential() 
model.add(Masking(mask_value=0., input_shape=(max_time, 24)))
model.add(LSTM(20, input_dim=24))
model.add(Dense(2))
model.add(Activation(activate))
model.compile(loss=weibull_loglik_discrete, optimizer=RMSprop(lr=.001))
model.fit(train_x, train_y, epochs=250, batch_size=2000, verbose=2, validation_data=(test_x, test_y))

Epoch 1/250
11/11 - 6s - loss: 19.9016 - val_loss: 7.9731
Epoch 2/250
11/11 - 4s - loss: 9.2199 - val_loss: 7.2690
Epoch 3/250
11/11 - 5s - loss: 8.7403 - val_loss: 7.2124
Epoch 4/250
11/11 - 5s - loss: 8.4702 - val_loss: 7.1638
Epoch 5/250
11/11 - 5s - loss: 8.2366 - val_loss: 7.1017
Epoch 6/250
11/11 - 4s - loss: 8.0246 - val_loss: 7.0276
Epoch 7/250
11/11 - 5s - loss: 7.8183 - val_loss: 6.9251
Epoch 8/250
11/11 - 5s - loss: 7.6070 - val_loss: 6.7731
Epoch 9/250
11/11 - 4s - loss: 7.3724 - val_loss: 6.5487
Epoch 10/250
11/11 - 4s - loss: 7.1216 - val_loss: 6.3170
Epoch 11/250
11/11 - 5s - loss: 6.8380 - val_loss: 5.9697
Epoch 12/250
11/11 - 4s - loss: 6.4631 - val_loss: 5.5627
Epoch 13/250
11/11 - 5s - loss: 6.1068 - val_loss: 5.3455
Epoch 14/250
11/11 - 4s - loss: 5.8991 - val_loss: 5.2762
Epoch 15/250
11/11 - 5s - loss: 5.8113 - val_loss: 5.2494
Epoch 16/250
11/11 - 5s - loss: 5.7505 - val_loss: 5.1988
Epoch 17/250
11/11 - 5s - loss: 5.8126 - val_loss: 5.3431
Epoch 18/250
11/11 - 5

In [40]:
"""
피팅한 모델로 예측하기. 
"""
test_predict = model.predict(test_x)
test_predict = np.resize(test_predict, (100, 2))
test_result = np.concatenate((test_y, test_predict), axis=1)
print(test_result)

[[112.           1.         198.66978455   4.11230183]
 [ 98.           1.         177.6108551    3.71354365]
 [ 69.           1.          92.75230408   1.73186457]
 [ 82.           1.         100.31937408   1.86831474]
 [ 91.           1.         117.66693115   2.18653274]
 [ 93.           1.          87.44530487   1.67709649]
 [ 91.           1.          95.11090851   1.80621028]
 [ 95.           1.         100.22809601   1.90449286]
 [111.           1.         176.24464417   3.57753158]
 [ 96.           1.          92.14980316   1.71814716]
 [ 97.           1.         151.41093445   2.99863911]
 [124.           1.          92.54492188   1.7398957 ]
 [ 95.           1.          95.70077515   1.82191455]
 [107.           1.         179.0954895    3.76012683]
 [ 83.           1.         154.0289917    3.15142584]
 [ 84.           1.          99.24423218   1.85282183]
 [ 50.           1.          96.34508514   1.81977439]
 [ 28.           1.         102.03434753   1.90822136]
 [ 87.    